# Reference model constraint

In [1]:
### check constraint value s in sbs code. 

#### how to spread values out laterally
    #### how to relax constraints laterally
    
    
#### how to change the number of layers




In [2]:
#import libraries
import pandas as pd
import numpy as np
import xarray as xr

# User requirement: enter file input information

In [3]:
## User requirement: enter the folder location where the .dat file is saved
folder= str(r"C:/Users/u67397/AnacondaProjects/aem/input_data/")

## User requirement: enter the name of the second .dat file (without the ".dat" text)
dat_name= str("AA140005_Line1080")

## User requirement: enter the name of the second .dat file (without the ".dat" text)
con_name= str("galeisbstdem")

# User requirement: enter reference model data variables

In [4]:
## User requirement: enter the reference model conductivity specified for result file
reference_EC1= 0.001

## User requirement: enter the number of layers result file
layers= 30

##User requirement: set constraint weighting. 1% =no constraint (e.g. reference model) and 100%= hard constraint at location
no_constraint=1
hard_constraint=10
lateral_constraint=3

In [5]:
#Enter column number within .dat file that contains the longitude and latitude. For example, the third column should be "3",
#please do not use programmer counting.
#coord- easting/northing
lat_column= 3
long_column= 4

##Other variables here

# User requirement: enter file output information

In [7]:
## User requirement: enter the folder location where to save files into

output_folder= str(r"C:/Users/u67397/AnacondaProjects/aem/output_data")

# Create array that contains contrained data

In [8]:
#load .dat file and then process
dat= pd.read_fwf(folder+dat_name+".dat", header=None) #load dat file
dat=dat.set_index([2,3], drop=False) #set coordinates as index
dat['all_sumcoord']=dat[2]+dat[3] #create a new column that combines coordinates
datlen= len(dat.index) #create variable of dat length

In [9]:
#extract thickness data from .dat file and then process
con = pd.read_csv (folder+con_name+".con", header=None) #import control file
con.replace(regex=True,to_replace=r'\t', value=r'', inplace=True)#remove tabs
tlayers=con.loc[con[0].str.contains('Thickness      = ')] #create a dataframe that contains layer thickness saved in confile (.con)
thicknessoflayers= tlayers.replace(regex=True,to_replace=r'Thickness      = ', value=r'')
thicknessoflayers=thicknessoflayers[0].str.split((' '),expand=True).replace('',np.nan)
thicknessoflayers.dropna(axis=1,inplace=True)
thicknessoflayers=thicknessoflayers.reset_index(drop=True).T.reset_index(drop=True).T
thicknessoflayers=thicknessoflayers.transpose().astype(float)
bottomlayer=thicknessoflayers[0].iloc[-1]
bottomlayer=np.array([bottomlayer])
bottomlayer=pd.DataFrame(bottomlayer)
thicknessoflayers=thicknessoflayers.append(bottomlayer).reset_index(drop=True)
thicknessoflayers=np.cumsum(thicknessoflayers)
thicknessoflayers=thicknessoflayers.transpose()
thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))
thicknessoflayers=pd.DataFrame(thicknessoflayers)

#thickness
thickness_1= thicknessoflayers[0:1]
thickness_1=np.array(thickness_1).squeeze()

numberoflocations=list(range(layers))
numberoflocations=np.asarray(numberoflocations)

In [10]:
#create dataframe of reference EC values
EC=pd.Series(reference_EC1)
EC=(pd.np.tile(EC, (datlen, layers)))
EC=pd.DataFrame(EC)

In [11]:
#load constraint file and process 
constraint_name=str("input_test") 
constraint_data= pd.read_csv(folder+constraint_name+".csv") #import control file
constraint_data['const_sumcoord']=constraint_data['longitude']+constraint_data['latitude']
constraint_data=constraint_data.set_index(['ID'], drop=False)

#create data array of reference constraint values
constraint_weighting=np.full((datlen,layers),no_constraint)
constraint_data

,ID,longitude,latitude,layer_top,layer_bottom,layer_EC,const_sumcoord
ID,,,,,,,
erin,erin,220955.6,6867443.6,0,10,0.05,7088399.2
rules,rules,220956.6,6867444.6,10,20,0.10,7088401.2


In [12]:
#coordinates
longitude_coords=pd.DataFrame(dat.iloc[:,(long_column -2):(long_column -1)])
longitude_coords=longitude_coords.transpose()
longitude_coords=np.array(longitude_coords).squeeze()

latitude_coords=pd.DataFrame(dat.iloc[:,(lat_column -2):(lat_column -1)])
latitude_coords=latitude_coords.transpose()
latitude_coords=np.array(latitude_coords).squeeze()
latitude_array=pd.np.tile(latitude_coords, ((len(numberoflocations), 1))).transpose()

In [13]:
matched=pd.DataFrame([0,0,0]).transpose().set_index([0])

for x in range(len(constraint_data)):
    g=(constraint_data.ID[x])
    h=(constraint_data.const_sumcoord[x])
    dat[str(g)]= np.abs(dat.all_sumcoord-h) 
    matched1=np.argmin(dat[str(g)])
    e_matched=matched1[0]
    n_matched=matched1[1]
    matched2=pd.DataFrame([g,float(e_matched),float(n_matched)]).transpose().set_index([0]).astype(float)
    matched=pd.concat([matched,matched2], axis=0)

matched=matched.rename(columns={1:'long_matched',2:'lat_matched'}).drop([0])

In [14]:
constraint_data2= pd.concat([constraint_data, matched],axis=1)
constraint_data2=constraint_data2.set_index(['long_matched','lat_matched'],drop=False)
constraint_data2

,,ID,longitude,latitude,layer_top,layer_bottom,layer_EC,const_sumcoord,long_matched,lat_matched
long_matched,lat_matched,,,,,,,,,
220955.6,6867443.6,erin,220955.6,6867443.6,0,10,0.05,7088399.2,220955.6,6867443.6
220958.6,6867443.7,rules,220956.6,6867444.6,10,20,0.10,7088401.2,220958.6,6867443.7


In [18]:
for y in range(len(constraint_data2)):
    EC_value1=constraint_data2['layer_EC'][y]
    zt=constraint_data2['layer_top'][y]
    zb=constraint_data2['layer_bottom'][y]
    long=float("{0:.2f}".format(constraint_data2['long_matched'][y]))
    lat=float("{0:.2f}".format(constraint_data2['lat_matched'][y]))    
    
    #Convert datasets into arrays 
    coords_da1={'longitude': longitude_coords, 'depth':thickness_1} #Create EC array
    ec_da = xr.DataArray(EC, dims=('longitude', 'depth'), coords=coords_da1)
    
    coords_da2={'longitude': longitude_coords, 'layers': numberoflocations}#Create thickness array
    thickness_da = xr.DataArray(thicknessoflayers, dims=('longitude', 'layers'), coords=coords_da2)
    
    coords_da3={'longitude': longitude_coords, 'depth': thickness_1}
    latitude_da = xr.DataArray(latitude_array, dims=('longitude','depth'), coords=coords_da3)
    
    coords_da4={'longitude': longitude_coords, 'depth': thickness_1}
    weighting_da = xr.DataArray(constraint_weighting, dims=('longitude','depth'), coords=coords_da4)
    
    dataset = xr.Dataset({'ec_dataset': ec_da, 'thickness_dataset': thickness_da, 'northing_dataset':latitude_da, 'weighting_dataset':weighting_da}) #Create combined EC and thickness array
    
    #Change EC and weighting values at certain depths
    array2=dataset.assign(ec=EC_value1).where((thickness_da>zt)&(thickness_da<zb)).sel(longitude=long)
    array2['ec']=array2.ec.fillna(reference_EC1)
    array3=np.array(array2['ec'])
    array4=dataset.assign(weighting=hard_constraint).where((thickness_da>zt)&(thickness_da<zb)).sel(longitude=long) 
    print()
    array4['weighting']=array4.weighting.fillna(no_constraint)
    array5=np.array(array4['weighting'])
    #Create array with updated EC and weighting values at certain depths
    ec_da.loc[long]=array3
    weighting_da.loc[long]=array5
    dataset = xr.Dataset({'ec_dataset': ec_da, 'thickness_dataset': thickness_da, 'latitude_dataset':latitude_da, 'weighting_dataset':weighting_da}) #Create combined EC and thickness array

#dataset.sel(longitude=220958.6)

In [17]:
# weighting_da.T
# # weighting_da.depth#.sel(longitude=220958.6)
# # weighting_da.sel(longitude=220958.6)
# weighting_da#.sel(longitude=220958.6)

In [17]:
# weighting_da= weighting_da.data[:,:]

In [18]:
# test=np.zeros((3,5))
# test[:2,2]=1    

In [19]:
# test=np.zeros((4,5))
# test[:2,2]=2 

# ix_x,ix_y = np.where(test>0) 
# print(ixx)
# print(ixy)
# test[ix,0:2] = 1
# test[ix,3:4] = 1
# print(test)

In [32]:
test=np.zeros((15,11))
test[:2,5]=hard_constraint 
# test[:2,10]=hard_constraint 

blah_max=np.linspace(hard_constraint,no_constraint,num=lateral_constraint,endpoint=False)
blah_min=np.linspace(no_constraint,hard_constraint,num=lateral_constraint,endpoint=False)

for row in test:
    if row.any()>0:
        ixx = np.where(row==hard_constraint) 
        print(ixx)
        ixx= int(ixx[0])
        
        ixx_min_down=ixx-lateral_constraint
        ixx_min_up=ixx
            
        ixx_max_down=ixx
        ixx_max_up=ixx+lateral_constraint
        row[ixx_min_down:ixx_min_up]=blah_min
#         row[ixx_max_down:ixx_max_up]=blah_max


np.place(test,test==0,1)

print (test)

(array([5], dtype=int64),)
(array([5], dtype=int64),)
[[  1.    2.8   4.6   6.4   8.2  10.    1.    1.    1.    1.    1. ]
 [  1.    2.8   4.6   6.4   8.2  10.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1. ]
 [  1.    1.    1.    1.    1.    1.    1.    1. 

In [167]:
#         for value in np.nditer(test,op_flags=['readwrite']):
#             if  value > 0:

In [88]:
##create function that 
def test()

SyntaxError: invalid syntax (<ipython-input-88-addedfe3037b>, line 2)

In [95]:
weighting_da

<xarray.DataArray (longitude: 20845, depth: 30)>
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ..., 
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])
Coordinates:
  * longitude  (longitude) float64 2.209e+05 2.209e+05 2.209e+05 2.209e+05 ...
  * depth      (depth) float64 3.0 6.3 9.92 13.92 18.32 23.16 28.48 34.32 ...

In [16]:
stop here

SyntaxError: invalid syntax (<ipython-input-16-f9ef205a8f26>, line 1)

In [18]:
dataset

<xarray.Dataset>
Dimensions:            (depth: 30, layers: 30, longitude: 20845)
Coordinates:
  * longitude          (longitude) float64 2.209e+05 2.209e+05 2.209e+05 ...
  * depth              (depth) float64 3.0 6.3 9.92 13.92 18.32 23.16 28.48 ...
  * layers             (layers) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...
Data variables:
    ec_dataset         (longitude, depth) float64 0.001 0.001 0.001 0.001 ...
    thickness_dataset  (longitude, layers) float64 3.0 6.3 9.92 13.92 18.32 ...
    latitude_dataset   (longitude, depth) int64 45492 45492 45492 45492 ...
    weighting_dataset  (longitude, depth) int32 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...

In [44]:
#create and save a .csv that contains depth of investigation data for input into Discover PA software
dat_test.to_csv(output_folder+'/depth_data.csv') #save doi data as csv

NameError: name 'dat_test' is not defined

In [ ]:
# #create subset of three dataframes and reformat

# #thickness
# thickness_1=thicknessoflayers
# thickness_2= thicknessoflayers[0:1]
# thickness_2=np.array(thickness_2).squeeze()

In [ ]:
# #create data array of reference constraint values
# constraint_weighting=np.full((datlen,layers),no_constraint)

In [ ]:
# dat_test=dat.set_index([2,3], drop=False)
# dat_test['all_sumcoord']=dat_test[2]+dat_test[3]

In [ ]:
# numberoflocations=list(range(layers))
# numberoflocations=np.asarray(numberoflocations)

In [ ]:
# EC_value1=constraint_data2['v'][0]
# zt=constraint_data2['zt'][0]
# zb=constraint_data2['zb'][0]
# easting=float("{0:.2f}".format(constraint_data2['E_matched'][0]))

In [ ]:
# coords_5=pd.np.tile(coords_4, ((len(coords_4), 1)))

In [ ]:
# #Convert datasets into arrays

# #Create EC array
# coords1={'easting':coords_2, 'depth':thickness_2}
# ec_da = xr.DataArray(EC, dims=('easting', 'depth'), coords=coords1,)

# #Create thickness array
# coords2={'easting': coords_2, 'layer': [1, 2, 3, 4, 5]}
# thickness_da = xr.DataArray(thicknessoflayers, dims=('easting', 'layer'), coords=coords2)

# #Create thickness array
# coords3={'northing': coords_4, 'depth': thickness_2}
# coords_da = xr.DataArray(coords_5, dims=('northing','depth'), coords=coords3)
# # coords3={'northing': coords_4, 'depth': thickness_2}
# # coords_da = xr.DataArray(thickness_1, dims=('northing','depth'), coords=coords3)


# #Create combined EC and thickness array
# dataset = xr.Dataset({'ec_dataset': ec_da, 'thickness_dataset': thickness_da, 'northing_dataset':coords_da})
# # dataset = xr.Dataset({'ec': ec_da, 'thickness_name': thickness_da, 'thickness_name2':coords_da})

# dataset

In [ ]:
# #Change EC values at certain depths
# array2=dataset.assign(ec=EC_value1).where((thickness_da>zt)&(thickness_da<zb)).sel(northing=northing).sel(easting=easting)
# array2['ec']=array2.ec.fillna(reference_EC1)
# array3=np.array(array2['ec'])

In [ ]:
# #Create array with updated EC values at certain depths
# ec_da.loc[easting]=array3
# dataset = xr.Dataset({'ec_dataset': ec_da, 'thickness_dataset': thickness_da, 'northing_dataset':coords_da}) #Create combined EC and thickness array
# dataset

In [ ]:
# alldata=alldata.set_index([2,3], drop=False)
# alldata['all_sumcoord']=alldata[2]+alldata[3]

In [ ]:
# #join datafile and layer thickness dataframes
# alldata=pd.concat([dat,thicknessoflayers,EC], axis=1, join='outer',ignore_index=True)


In [ ]:
# coords_m=dat.iloc[:,1:3]
# EC_1= xr.Dataset(EC.iloc[0:9,0:9])
# thickness_m=xr.Dataset(thicknessoflayers.iloc[0:9,0:9])
# dat_m=xr.Dataset(dat.iloc[0:9,0:9])


In [ ]:
# dataset = xarray.Dataset({'ec': ec_da, 'thickness_name': thickness_da})
# dataset.ec

In [ ]:
# coords={'location': coords_test, 'layer': [1, 2, 3, 4, 5]}
# thickness_da = xarray.DataArray(thickness_m, dims=('location', 'layer'), coords=coords)

In [ ]:
# thickness_test= thickness_m[0:1]
# thickness_test=np.array(thickness_test).squeeze()
# thickness_test

In [ ]:
# coords_test= coords_m.transpose()
# coords_test=np.array(coords_test).squeeze()

In [ ]:
# dataset.sel(location=220926.1).update({dataset.ec:('ec2',[0.05,  0.05,  0.01,  0.01,  0.01])}, inplace=True)
# dataset.sel(location=220926.1).update({'ec':('ec2',[0.05,  0.05,  0.01,  0.01,  0.01])}, inplace=True)
#dataset.location[0]#=dataset.sel(location=220926.1)#.update({dataset.ec:('ec2',[0.05,  0.05,  0.01,  0.01,  0.01])}, inplace=True)
# dataset.loc['location',['220926.1']]

#dataset.update({'ec':('ec',[1,2,3])})
#dataset.update({(dataset.sel(location=220926.1).ec):(1)})
# dataset.ec

In [ ]:
# test2= dataset.sel(location=220926.1)
# test2=test2.thickness_name<10#.ec
# test2

In [ ]:
# dataset.ec.isel_points(location=(dataset.thickness_name < 10).location)

In [ ]:
# dataset.where(dataset.sel(location=51).thickness_name < 10).fillna(100)

In [ ]:
# dataset.location[:] = [5, 6, 7, 8, 9]

In [ ]:

#coords_m=pd.DataFrame(dat.iloc[0:5,2:4])

# test=pd.concat([coords_m,thickness_m,EC_m],axis=1,ignore_index=True).set_index([0,1])
# test=pd.concat([test,constraint_data2],axis=1)
# test 


# thickness_m=pd.concat([coords_m,thickness_m],axis=1,ignore_index=True)#.set_index([0,1])
# array=np.array([[coords_m],[thickness_m],[EC_m]])
# array.shape

In [ ]:
# for row in constraint_data:
#     print(row)
    #sample[str(row)]= 0

#print(sample)
    
    
#     print('loop1_start')
#     print(row)
#     print('loop1_ends')
#     for blah in constraint_data['const_sumcoord']:
#         print('loop2_starts')
#         print (blah)
#         print('loop2_ends')
# #         while x>0:
#         for rah in sample:
#             sample[str(row)]=np.abs(test.all_sumcoord-blah)
# #                 x=x-1
#             print(sample)
                

In [ ]:
## find coordinate
## enter layer thickness (e.g 3)
## extrapolate between the points to maximum layers (e.g. 30)

In [ ]:
# array=np.array([[dat],[thicknessoflayers],[EC]],ndmin=3)
# array.shape
# #test=np.ndarray(array)
# array[[0,0,0],[[0,0,0],[0,0,0]]]

In [ ]:
# #create a dataframe that contains layer thickness saved in confile (.con)
# tlayers=con.loc[con[0].str.contains('Thickness      = ')]
# thicknessoflayers= tlayers.replace(regex=True,to_replace=r'Thickness      = ', value=r'')
# thicknessoflayers=thicknessoflayers[0].str.split((' '),expand=True).replace('',np.nan)
# thicknessoflayers.dropna(axis=1,inplace=True)
# thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))
# #thicknessoflayers=pd.DataFrame(thicknessoflayers)
# thicknessoflayers

In [ ]:
# thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))


In [ ]:
# def coord(easting,northing):
#     easting=int(easting)
#     northing=int(northing)
#     print(type(easting))

# coord('1','2')

In [ ]:
# def print_text():
#     print("complete")

In [ ]:
# a = [[1, 2, 3, 4], [5, 6], [7, 8, 9]]
# for i in range(len(a)):
#     for j in range(len(a[i])):
#         print(a[i][j], end=' ')
#     print()

In [ ]:
# EC_x=xr.Dataset([EC_m],[thickness_m])
# EC_x